In [1]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.0 MB/s eta 0:00:00
  Using cached accelerate-0.30.0-py3-none-any.whl (302 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvi

## 0. Imports


In [2]:
# Imports
!pip install contractions
import pandas as pd
import spacy
from nltk.corpus import stopwords
import contractions
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW
)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy as sp
import numpy as np
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 13.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Ejercicio


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio debeis entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso podréis centraros en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.


**Nota 1**: Este ejercicio requiere el uso de las GPUs de Google Colab. Este Colab debería estar preconfigurado para ejecutarse en GPU, pero si tuviera problemas en la ejecución que me contacte a través del Moodle para buscar soluciones alternativas.

## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [3]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

In [4]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preprocesado y Normalización

In [5]:
def eliminar_espacios(text):
  return " ".join(text.split())

def texto_to_lower(text):
  return text.lower()

def preprocess_text(text: str) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)

    # remove stopwords
    tokens = nltk.word_tokenize(text)
    text = " ".join(tokens)
    text = text.lower().strip()

    return text

def replace_contractions(text):
  return contractions.fix(text, slang=True)

tweet_tokenizer = TweetTokenizer()
def normalizar_tokens(documento):
  # Dividir en palabras
  palabras = tweet_tokenizer.tokenize(documento)
  # Reemplazar URLs, menciones de Twitter y número por tokens
  for i in range(len(palabras)):
    if palabras[i].startswith("http://") or palabras[i].startswith("https://") or palabras[i].startswith('www.'):
      palabras[i] = "URL"
    elif palabras[i].startswith("@"):
      palabras[i] = "MENTION"
    elif palabras[i].isdigit():
      palabras[i] = "NUM"

    # Unir palabras de nuevo en un documento modificado
    documento_modificado = ' '.join(palabras)
    return documento_modificado

In [6]:
# Crear columna char_len
dataset['char_len'] = dataset['tweet_text'].apply(lambda x: len(x))
# Quitar duplicados
dataset = dataset.loc[~dataset['tweet_text'].duplicated()]

En esta celda vamos a normalizar el texto, quitandole los espacios, pasando el texto a minuscula, expandiendo contracciones gramaticales y reemplazando vinculos urls, mencions y digitos por su respectiva clave.

In [7]:
from tqdm.autonotebook import tqdm
tqdm.pandas()
# Eliminar Espacios
dataset["texto_normalizado"] = dataset["tweet_text"].progress_apply(lambda x: eliminar_espacios(x))
# Pasar a minuscula
dataset["texto_normalizado"] = dataset["texto_normalizado"].progress_apply(lambda x: texto_to_lower(x))
# Expandir contracciones
dataset["texto_normalizado"] = dataset["texto_normalizado"].progress_apply(lambda x: replace_contractions(x))
# Lidiar con Https: @, digitos y www
dataset["texto_normalizado"] = dataset["texto_normalizado"].progress_apply(lambda x: normalizar_tokens(x))
# Longitud de carácteres
dataset["char_len_normalizado"] = dataset["texto_normalizado"].apply(lambda x: len(x))

  0%|          | 0/46017 [00:00<?, ?it/s]

  0%|          | 0/46017 [00:00<?, ?it/s]

  0%|          | 0/46017 [00:00<?, ?it/s]

  0%|          | 0/46017 [00:00<?, ?it/s]

Con el proposito de reducir la dimensionalidad de la matriz vamos a lematizar y quitar stopwords en la columna del texto normalizado, utilizando el encoder web en ingles de spacy, por ultimo vamos a reasignar este texto lematizado a la misma columna de texto_normalizado

In [8]:
# Lematización de carácteres
en_stopwords = stopwords.words('english')
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner']) # Tokenizador en ingles
def lematizar_eliminacion_tokens(texto):
  doc = nlp(texto) # Tokenizar documento
  lemas = [token.lemma_ for token in doc] # Extraer lemma de token
  tokens_filtrados = [token for token in lemas if token.isalpha() and token.lower() not in en_stopwords] # Extraer token si es minuscula y si se compone de texto, excluir si es parte de stopwords
  texto_procesado = ' '.join(tokens_filtrados) # Unir texto
  return texto_procesado

dataset["texto_normalizado"] = dataset["texto_normalizado"].progress_apply(lambda x: lematizar_eliminacion_tokens(x))

  0%|          | 0/46017 [00:00<?, ?it/s]

In [9]:
# Catacteristicas adicionales
def token_appearance(document, token = "MENTION"):
  return len([x for x in document.split() if x == token])

## Ingenieria de caracteristicas
dataset["num_mention"] = dataset["texto_normalizado"].progress_apply(lambda x: token_appearance(x, token="MENTION"))
dataset["num_URLS"] = dataset["texto_normalizado"].progress_apply(lambda x: token_appearance(x, token="URL"))
dataset["num_NUM"] = dataset["texto_normalizado"].progress_apply(lambda x: token_appearance(x, token="NUM"))

  0%|          | 0/46017 [00:00<?, ?it/s]

  0%|          | 0/46017 [00:00<?, ?it/s]

  0%|          | 0/46017 [00:00<?, ?it/s]

## 4. Vectorización, Entrenamiento y Evaluación


In [12]:
# Guardar variables independientes y variable dependiente
text = dataset["texto_normalizado"].values
num_mention = dataset["num_mention"].values
num_urls = dataset["num_URLS"].values
num_num = dataset["num_NUM"].values
label = dataset["cyberbullying"].values

División de dataset en entrenamiento y validación.

In [13]:
train_texts, test_texts, train_labels, test_labels = train_test_split(text, label, test_size=.25, random_state=0,
                                                    stratify = label)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=0,stratify = train_labels)

In [14]:
# Elegir el modelo preentrenado y un tokenizador
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Dado que los modelo 'bert-base-uncased' utiliza una estructura especifica para el entrenamiento de datos, no me fue posible adecuar la estructura de TfidfVectorizer con un formato compatible para la aplicación de la libreria transfomers en el proceso de clasificicación, por lo que se utiliza la siguiente formula previamente establecida en el curso para un preproesado con dicha compatibilidad.

In [15]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [16]:
max_length = 280  # Puedes ajustar esto según tus necesidades

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)


Es necesario predefinir la longitud máxima en tokens, por lo que se usarán la configuración elegida en los ejercicios. En este caso dado que se tiene mucho más texto se utilizaran batch sizes más grandes y el max_seq_length que se utilizará será el de 280.

In [30]:
max_seq_length = 280 #@param {type: "integer"}
train_batch_size =  40#@param {type: "integer"}
eval_batch_size = 40 #@param {type: "integer"}
test_batch_size = 40 #@param {type: "integer"}

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "No_Cyberbullying", 1: "Cyberbullying"}
label2id = {"No_Cyberbullying": 0, "Cyberbullying": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2, id2label=id2label, label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
import accelerate
import numpy as np
import evaluate

training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

In [ ]:
# 4 Epocas
# Batch size 40
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.257400,0.241023,{'accuracy': 0.8852672750977836},{'f1': 0.9320871205625108}
2,0.212400,0.239674,{'accuracy': 0.8843980877879183},{'f1': 0.9306689834926152}
3,0.158100,0.280634,{'accuracy': 0.8870056497175142},{'f1': 0.9330357142857143}


Trainer is attempting to log a value of "{'accuracy': 0.8852672750977836}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9320871205625108}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8843980877879183}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9306689834926152}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8870056497175142}" of type <clas

Dado a que la duración del entrenamiento, excedio el tiempo permitido para la ejecucción de una celda en google collab, no fue posible acabar la última epoca de entrenamiento. Sin embargo, esto no impide hacer la comparación entre nuestro modelo logístico del ejercicio anterior y nuestro desempeño de la red hasta la tercer epoca.

 - Accuracy : 88.700 (Red Neuronal)
 - Accuracy : 85.5600 (Regresión Logistica)

 Es claro que la red neuronal tiene un desmpeño mayor que muy seguramente pueda ser mejorado aun más, con un mayor número de epocas y un ajuste hiperparámetrico. No obstante, no puedo lograr esto en el ambiente actual, no por falta de interés en el tema, sino porque google collab se sigue desconectando y mi pc local no aguanta el código.

 Algo interesante a recalcar, es que las epocas no estan disminuyendo el grado de error de la red en el ejercicio de clasificación, no se si sea porque la red ya esta bien estructurada y los pesos sinapticos esten ya bien definidos para la ejecucción de tareas relacionadas al lenguaje, dado que estamos utilizando un modelo preentrenado o bien simplemente el descenso de gradiente no sería notorio hasta muchas más epocas.

 De cualquier forma es un desempeño bastante bueno utilizando 1 sola epoca.